In [1]:
import cv2
from matplotlib import pyplot as plt
import numpy as np 
import random
import math
imgs = ["baboon.jpg","SAFlag.jpg"]
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

In [2]:
def random_centers(img,k): 
    centers = []
    for i in range(k):
        pixel = img[random.randrange(img.shape[0]),random.randrange(img.shape[1])]
        centers.append([pixel[0],pixel[1],pixel[2]])
    return centers

In [3]:
def voronoi(imgss,centerss,segmented_val,cluster_arrs):
    sums = [[0,0,0] for i in range(len(cluster_arrs))]
    lengths = [0 for i in range(len(cluster_arrs))]
    means = []
    for i in range(0,imgss.shape[0]): 
        for j in range(0,imgss.shape[1]):
            point = imgss[i,j]
            pxl = [point[0],point[1],point[2]]
            eucld_distances = []
            for center_val in centerss:
                center = [center_val[0],center_val[1],center_val[2]]
                val = math.sqrt((center[0]- pxl[0])**2  + (center[1]- pxl[1])**2  + (center[2]- pxl[2])**2)
                eucld_distances.append(val)
            min_val = min(eucld_distances)
            min_index = eucld_distances.index(min_val)
            cluster_arrs[min_index].append(pxl)
            #Calculate sums and lengths as you go along such that cycles are not wastes
            sums[min_index] = [sums[min_index][0] + pxl[0],sums[min_index][1]+ pxl[1],sums[min_index][2] + pxl[2]]
            lengths[min_index] = lengths[min_index] + 1
            segmented_val[i,j] = min_index
    means = [[sums[i][0]/lengths[i],sums[i][1]/lengths[i],sums[i][2]/lengths[i]] for i in range(len(sums))]
    
    return [segmented_val,cluster_arrs,means]

In [4]:
def kmeans(img,k):
    img_c  = img.copy()
    regions = [[] for i in range(k)]
    prev_centers = [[] for i in range(k)]
    centers = random_centers(img_c,k)

    #pixel labels
    segmented_vals = np.zeros((img_c.shape[0],img_c.shape[1]))
    difference = False
    while not(difference):
        segmented_vals,regions,means = voronoi(img_c,centers,segmented_vals,regions)
        counter = 0
        for region in regions: 
            if region != []: 
                prev_centers[counter] = centers[counter]
                centers[counter] = means[counter]
                counter = counter + 1 
                
        length = len(centers)
        length_rgb = len(centers[0])
        count_true = 0
        count_false = 0
        #checks the difference between previous center values and updated and if the value is less than .5
        #A True flag will be added to indicate that number of RGB values that have converged
        for i in range(length):
            for j in range(length_rgb):
                if abs(centers[i][j] - prev_centers[i][j]) < .5:
                    count_true += 1
                else: 
                    count_false += 1
        # If there is an RGB values that has not converged then the algorithm loops 
        # Other wise if 1 or less RGB values the estimation is close enough
        # This is done to reduce computation time when the bottelneck is one RGB value that has not converged, 
        # especially after multiple iterations where that value is probably very close to the tolerence.
        if count_false > 0: 
            difference = False
        else: 
            difference = True            
    
    segmented_img = np.zeros(img_c.shape, np.uint8)
    for i in range(segmented_img.shape[0]):
        for j in range(segmented_img.shape[1]): 
            segmented_img[i,j] = centers[int(segmented_vals[i,j])]
    return segmented_img

Please set the value of K in the trackbar and then click on the image to segment the image. This was done such that the algorithm doesnt start immediatly after setting K.

In [5]:
for img in imgs: 
    
    def onMouse(event,x,y,flags, param): 
        if event == cv2.EVENT_LBUTTONDOWN:
            k = cv2.getTrackbarPos('k', 'Color Quantization')
            if k == 0: 
                k = 1
            segmented_img = kmeans(img,k)
            cv2.imshow("Color Quantization",segmented_img)

    def changeK(val):
        k = val
      

    temp = cv2.imread(img)
    img = temp.copy()
    cv2.namedWindow('Color Quantization')
    cv2.createTrackbar('k', 'Color Quantization',0,255,changeK)
    cv2.setMouseCallback('Color Quantization', onMouse)
    cv2.imshow('Color Quantization', img)
   
    
    key = cv2.waitKey(10)
    
    
    while(key != ord('x')): 
        key = cv2.waitKey(0)
    cv2.destroyAllWindows()            